# **Import**

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style="darkgrid")
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score

from google.colab import drive
drive.mount('***') # Fill in the path to the folder where the data is stored

# **Data Cleaning**

In [ ]:
def clean_dataframe(df):
    # Fill NaN values with '0' in the df
    df.fillna('0', inplace=True)

    # Apply encoding fix to specified columns. This is to handle names or clubs with accents or special characters
    encoding_columns = ['Name', 'Club']
    for column in encoding_columns:
        df[column] = df[column].apply(lambda x: try_encoding_fix(x) if isinstance(x, str) else x)

    # Convert specified columns to string
    columns_to_string = ['Name', 'Best Pos', 'Club', 'Division']
    df[columns_to_string] = df[columns_to_string].astype('string')

    # Define columns to convert to numbers and clean non-numeric characters
    columns_to_numbers = [
        'Tck/90', 'Shot/90', 'ShT/90', 'Shots Outside Box/90', 'Shts Blckd/90',
        'Pr passes/90', 'Pres C/90', 'Pres A/90', 'Poss Won/90', 'Poss Lost/90',
        'OP-KP/90', 'OP-Cr %', 'NP-xG/90', 'K Tck/90', 'K Ps/90', 'K Hdrs/90',
        'Int/90', 'Hdr %', 'Gls/90', 'xG/90', 'xA/90', 'Drb/90', 'Cr C/A', 'Clr/90',
        'Ch C/90', 'Blk/90', 'Asts/90', 'Aer A/90', 'Av Rat'
    ]

    # Clean and convert columns to numbers
    df[columns_to_numbers] = df[columns_to_numbers].apply(lambda x: x.str.replace('-', '').str.replace('[^0-9.-]', '', regex=True).replace('', np.nan).astype(float).fillna(0))

    # Process 'Mins' separately to convert to integer
    df['Mins'] = df['Mins'].str.replace('-', '').str.replace('[^0-9]', '', regex=True).replace('', '0').astype(int)

    # Drop Inf column that is not needed but is automatically created via the in-game view
    df.drop(['Inf'], axis=1, inplace=True, errors='ignore')

    # Function to group positions
    def group_position(value):
        position_map = {
            'GK': 'Goalkeeper', 'D (R)': 'Full Back', 'D (L)': 'Full Back', 'D (C)': 'Centre Back',
            'WB (R)': 'Wing Back', 'WB (L)': 'Wing Back', 'DM': 'Defensive Midfielder',
            'M (R)': 'Winger', 'M (L)': 'Winger', 'M (C)': 'Central Midfielder',
            'AM (R)': 'Wide Attacking Midfielder', 'AM (L)': 'Wide Attacking Midfielder',
            'AM (C)': 'Central Attacking Midfielder', 'ST (C)': 'Striker'
        }
        return position_map.get(value, 'Unknown')

    # Apply the function to the 'Best Pos' column
    df['Best Pos'] = df['Best Pos'].apply(group_position)

    # Rename columns for clearer column names
    new_column_names = {
        'Best Pos': 'Position', 'Av Rat': 'Average Rating', 'Mins': 'Minutes Played',
        'Aer A/90': 'Aerial Challenges Attempted per 90', 'Asts/90': 'Assists per 90',
        'Blk/90': 'Blocks per 90', 'Ch C/90': 'Chances Created per 90', 'Clr/90': 'Clearances per 90',
        'Cr C/A': 'Cross Completion %', 'Drb/90': 'Dribbles per 90', 'xA/90': 'Expected Assists per 90',
        'xG/90': 'Expected Goals per 90', 'Gls/90': 'Goals per 90', 'Hdr %': 'Headers Won %',
        'Int/90': 'Interceptions per 90', 'K Hdrs/90': 'Key Headers per 90', 'K Ps/90': 'Key Passes per 90',
        'K Tck/90': 'Key Tackles per 90', 'NP-xG/90': 'Non-Penalty xG per 90', 'OP-Cr %': 'Open-Play Cross Completion %',
        'OP-KP/90': 'Open-Play Key Passes per 90', 'Poss Lost/90': 'Possession Lost per 90',
        'Poss Won/90': 'Possession Won per 90', 'Pres A/90': 'Pressures Attempted per 90',
        'Pres C/90': 'Pressures Completed per 90', 'Pr passes/90': 'Progressive Passes per 90',
        'Shts Blckd/90': 'Shots Blocked per 90', 'Shots Outside Box/90': 'Shots Outside Box per 90',
        'ShT/90': 'Shots on Target per 90', 'Shot/90': 'Shots per 90', 'Tck/90': 'Tackles Won per 90'
    }
    df.rename(columns=new_column_names, inplace=True)

    return df

def try_encoding_fix(value):
    try:
        return value.encode('latin1').decode('utf-8')
    except UnicodeDecodeError:
        try:
            return value.encode('windows-1252').decode('utf-8')
        except UnicodeDecodeError:
            return value

# Adding a season column to the df based on the file name
def add_season_column(df, file_name):
    season_year = file_name
    df['Season'] = season_year
    return df

# Function to convert HTML files to CSV files, clean the data, and add a 'Season' column
def convert_html_to_csv_with_cleaning(start_year, end_year):
    for year in range(start_year, end_year + 1):
        html_file_name = f'{year}.html'
        try:
            tables = pd.read_html(html_file_name)
            df = tables[0]

            # Clean the df
            df_cleaned = clean_dataframe(df)

            # Add the 'Season' column using the year as the season identifier
            df_with_season = add_season_column(df_cleaned, str(year))

            csv_file_name = f'season_{year}.csv'
            df_with_season.to_csv(csv_file_name, index=False)
            print(f'Converted {html_file_name} to {csv_file_name} with cleaning and added season')
        except Exception as e:
            print(f'Failed to process {html_file_name}: {e}')

# Call the function to convert and clean HTML files from 2024.html to 2038.html, adding the 'Season' column
convert_html_to_csv_with_cleaning(2024, 2038)

dfs = []

for year in range(2024, 2039):  # 2039 is exclusive, so it goes up to 2038
    file_name = f'season_{year}.csv'
    df = pd.read_csv(file_name)
    dfs.append(df)

# Concatenate all the dataframes into one final dataframe
final_df = pd.concat(dfs, ignore_index=True)

path_in_drive = '***' # Fill in the path to the folder where the data is stored

final_df.to_csv(path_in_drive, index=False)

print('All seasons concatenated into one CSV file.')

# **Positional DF's**

Divide the main df into positional df to be used in the model

In [ ]:
# Identify all unique positions
unique_positions = final_df['Position'].unique()

# Dictionary to store dfs divided by position but only include players with more than 900 minutes played
dfs_by_position = {}

for position in unique_positions:
    # Filter final_df for each position and for Minutes Played > 900 then store in the dictionary. Helps keep ratings more accurate
    dfs_by_position[position] = final_df[(final_df['Position'] == position) & (final_df['Minutes Played'] > 900)]

# Assuming your Google Drive is mounted and dfs_by_position is ready
for position, df in dfs_by_position.items():
    # Define the path in Google Drive where you want to save the CSV
    path_in_drive = f'***/{position}.csv' # Fill in the path to the folder where the data is stored

    # Save the DataFrame to a CSV file in the specified path
    df.to_csv(path_in_drive, index=False)

    print(f'{position}s with more than 900 minutes played saved to Google Drive.')

# **Model Preperation**

Final preperation of the data for the model

In [ ]:
# Function to double check & filter players with more than 900 minutes played
def filter_players(df):
    return df.query("`Minutes Played` > 900")

# Load the divided dfs back in from the CSV files saved to the Drive
full_backs = pd.read_csv('***/Full Back.csv')
wing_backs = pd.read_csv('***/Wing Back.csv')
centre_backs = pd.read_csv('***/Centre Back.csv')
central_attacking_midfielders = pd.read_csv('***/Central Attacking Midfielder.csv')
strikers = pd.read_csv('***/Striker.csv')
defensive_midfielders = pd.read_csv('***/Defensive Midfielder.csv')
central_midfielders = pd.read_csv('***/Central Midfielder.csv')
wide_attacking_midfielders = pd.read_csv('***/Wide Attacking Midfielder.csv')
wingers = pd.read_csv('***/Winger.csv')
goalkeepers = pd.read_csv('***/Goalkeeper.csv')

# Apply the filter_players function to each DataFrame
full_backs_filtered = filter_players(full_backs)
wing_backs_filtered = filter_players(wing_backs)
centre_backs_filtered = filter_players(centre_backs)
central_attacking_midfielders_filtered = filter_players(central_attacking_midfielders)
strikers_filtered = filter_players(strikers)
defensive_midfielders_filtered = filter_players(defensive_midfielders)
central_midfielders_filtered = filter_players(central_midfielders)
wide_attacking_midfielders_filtered = filter_players(wide_attacking_midfielders)
wingers_filtered = filter_players(wingers)
goalkeepers_filtered = filter_players(goalkeepers)

# Columns to keep for the model
columns_to_keep = [
        'Aerial Challenges Attempted per 90',
        'Assists per 90',
        'Blocks per 90',
        'Chances Created per 90',
        'Clearances per 90',
        'Cross Completion %',
        'Dribbles per 90',
        'Expected Assists per 90',
        'Expected Goals per 90',
        'Goals per 90',
        'Headers Won %',
        'Interceptions per 90',
        'Key Headers per 90',
        'Key Passes per 90',
        'Key Tackles per 90',
        'Non-Penalty xG per 90',
        'Open-Play Cross Completion %',
        'Open-Play Key Passes per 90',
        'Possession Lost per 90',
        'Possession Won per 90',
        'Pressures Attempted per 90',
        'Pressures Completed per 90',
        'Progressive Passes per 90',
        'Shots Blocked per 90',
        'Shots Outside Box per 90',
        'Shots on Target per 90',
        'Shots per 90',
        'Tackles Won per 90',
        'Average Rating'
    ]

full_backs_model = full_backs_filtered[columns_to_keep]
wing_backs_model = wing_backs_filtered[columns_to_keep]
centre_backs_model = centre_backs_filtered[columns_to_keep]
central_attacking_midfielders_model = central_attacking_midfielders_filtered[columns_to_keep]
strikers_model = strikers_filtered[columns_to_keep]
defensive_midfielders_model = defensive_midfielders_filtered[columns_to_keep]
central_midfielders_model = central_midfielders_filtered[columns_to_keep]  # Corrected
wide_attacking_midfielders_model = wide_attacking_midfielders_filtered[columns_to_keep]
wingers_model = wingers_filtered[columns_to_keep]
goalkeepers_model = goalkeepers_filtered[columns_to_keep]

# Save the position specifc dfs for the model as CSV files to the specified Drive path
full_backs_model.to_csv('***/full_backs_model.csv', index=False)
wing_backs_model.to_csv('***/wing_backs_model.csv', index=False)
centre_backs_model.to_csv('***/centre_backs_model.csv', index=False)
central_attacking_midfielders_model.to_csv('***/central_attacking_midfielders_model.csv', index=False)
strikers_model.to_csv('***/strikers_model.csv', index=False)
defensive_midfielders_model.to_csv('***/defensive_midfielders_model.csv', index=False)
central_midfielders_model.to_csv('***/central_midfielders_model.csv', index=False)
wide_attacking_midfielders_model.to_csv('***/wide_attacking_midfielders_model.csv', index=False)
wingers_model.to_csv('***/wingers_model.csv', index=False)
goalkeepers_model.to_csv('***/goalkeepers_model.csv', index=False)

# **Model & R2 Scores**

In [ ]:
class FeatureImportanceModel:
    def __init__(self, name, dataframe, target_column, importance_threshold):
        self.name = name
        self.dataframe = dataframe.copy()
        self.target_column = target_column
        self.importance_threshold = importance_threshold
        self.model = LinearRegression()
        self.selected_features = None
        self.importance_df = None
        self.new_importance_df = None

    # Function to precprocess the data
    def preprocess_data(self):
        X = self.dataframe.drop(self.target_column, axis=1)
        y = self.dataframe[self.target_column]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Impute missing values with the mean and scale the data. Fail safe in case of missing values missed in the cleaning process
        imputer = SimpleImputer(strategy='mean')
        X_train_imputed = imputer.fit_transform(X_train)
        X_test_imputed = imputer.transform(X_test)

        # Scale the data using StandardScaler
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train_imputed)
        X_test_scaled = scaler.transform(X_test_imputed)

        return X_train_scaled, X_test_scaled, y_train, y_test, X.columns

    # Function to train the initial model and print the R-Squared score for each poisition
    def train_initial_model(self):
        X_train_scaled, X_test_scaled, y_train, y_test, feature_names = self.preprocess_data()
        self.model.fit(X_train_scaled, y_train)
        y_pred = self.model.predict(X_test_scaled)
        r2 = r2_score(y_test, y_pred)
        print(f'{self.name} - Initial R-Squared:', r2)

        # Get the feature importances using the absolute value of the coefficients
        feature_importances = np.abs(self.model.coef_)
        self.importance_df = pd.DataFrame({
            'Feature': feature_names,
            'Importance': feature_importances
        }).sort_values(by='Importance', ascending=False)

    # Function to select important features based on the importance threshold as established by feature importance
    def select_important_features(self):
        self.selected_features = self.importance_df[self.importance_df['Importance'] > self.importance_threshold]['Feature'].tolist()

    # Function to train the model now with the selected features and print the R-Squared score
    def train_selected_model(self):
        X = self.dataframe[self.selected_features]
        y = self.dataframe[self.target_column]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Impute missing values with the mean and scale the data. Fail safe in case of missing values missed in the cleaning process
        imputer = SimpleImputer(strategy='mean')
        X_train_imputed = imputer.fit_transform(X_train)
        X_test_imputed = imputer.transform(X_test)

        # Scale the data using StandardScaler
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train_imputed)
        X_test_scaled = scaler.transform(X_test_imputed)

        self.model.fit(X_train_scaled, y_train)
        y_pred = self.model.predict(X_test_scaled)
        r2 = r2_score(y_test, y_pred)
        print(f'\n{self.name} - R-Squared with Selected Features:', r2)

        new_feature_importances = np.abs(self.model.coef_)
        self.new_importance_df = pd.DataFrame({
            'Feature': self.selected_features,
            'Importance': new_feature_importances
        }).sort_values(by='Importance', ascending=False)

    # Function to run the model
    def run(self):
        self.train_initial_model()
        self.select_important_features()
        self.train_selected_model()

    # Function to save the feature importance dfs to a CSV file for future use
    def save_feature_importance(self, filename):
        path = f'***/{filename}' # Fill in the path to the folder where the data is stored
        self.new_importance_df.to_csv(path, index=False)
        print(f"Saved feature importance for {self.name} to {filename}")

# Load the dfs for each position
centre_backs = pd.read_csv('***/centre_backs_model.csv')
full_backs = pd.read_csv('***/full_backs_model.csv')
wing_backs = pd.read_csv('***/wing_backs_model.csv')
defensive_midfielders = pd.read_csv('***/defensive_midfielders_model.csv')
central_midfielders = pd.read_csv('***/central_midfielders_model.csv')
central_attacking_midfielders = pd.read_csv('***/central_attacking_midfielders_model.csv')
wingers = pd.read_csv('***/wingers_model.csv')
wide_attacking_midfielders = pd.read_csv('***/wide_attacking_midfielders_model.csv')
strikers = pd.read_csv('***/strikers_model.csv')

# Define the models with different thresholds. These were determined through trial and error
centre_backs_model = FeatureImportanceModel('Centre Backs', centre_backs, 'Average Rating', 0.0018)
full_backs_model = FeatureImportanceModel('Full Backs', full_backs, 'Average Rating', 0.002)
wing_backs_model = FeatureImportanceModel('Wing Backs', wing_backs, 'Average Rating', 0.0015)
defensive_midfielders_model = FeatureImportanceModel('Defensive Midfielders', defensive_midfielders, 'Average Rating', 0.0025)
central_midfielders_model = FeatureImportanceModel('Central Midfielders', central_midfielders, 'Average Rating', 0.002)
central_attacking_midfielders_model = FeatureImportanceModel('Central Attacking Midfielders', central_attacking_midfielders, 'Average Rating', 0.0022)
wingers_model = FeatureImportanceModel('Wingers', wingers, 'Average Rating', 0.0018)
wide_attacking_midfielders_model = FeatureImportanceModel('Wide Attacking Midfielders', wide_attacking_midfielders, 'Average Rating', 0.002)
strikers_model = FeatureImportanceModel('Strikers', strikers, 'Average Rating', 0.002)

# Run the models
centre_backs_model.run()
full_backs_model.run()
wing_backs_model.run()
defensive_midfielders_model.run()
central_midfielders_model.run()
central_attacking_midfielders_model.run()
wingers_model.run()
wide_attacking_midfielders_model.run()
strikers_model.run()

# Save feature importance dfs to CSV files
centre_backs_model.save_feature_importance('centre_backs_importance.csv')
full_backs_model.save_feature_importance('full_backs_importance.csv')
wing_backs_model.save_feature_importance('wing_backs_importance.csv')
defensive_midfielders_model.save_feature_importance('defensive_midfielders_importance.csv')
central_midfielders_model.save_feature_importance('central_midfielders_importance.csv')
central_attacking_midfielders_model.save_feature_importance('central_attacking_midfielders_importance.csv')
wingers_model.save_feature_importance('wingers_importance.csv')
wide_attacking_midfielders_model.save_feature_importance('wide_attacking_midfielders_importance.csv')
strikers_model.save_feature_importance('strikers_importance.csv')

# **Adjusted Rating**

In [ ]:
# Load the data
df = pd.read_csv('***/final_df.csv')

# Filter out Goalkeepers and Unknown positions from the dataframe
df = df[~df['Position'].isin(['Goalkeeper', 'Unknown'])]

# Read the world DataFrame from an HTML file
world = pd.read_html('world.html')[0]

# Create the division_ranks dictionary based on the order of the leagues in the 'world' DataFrame
division_ranks = {league: rank + 1 for rank, league in enumerate(world['Name'])}
print("Division Ranks:")
for division, rank in division_ranks.items():
    print(f"{division}: {rank}")

def calculate_division_weight(rank):
    """
    Calculate the division weight based on the division rank.
    Lower rank numbers (top of the list) represent better divisions and should have higher weights.
    """
    return 1 / (rank ** 0.5)  # Using square root to reduce the impact of rank differences

# Function to calculate ratings for a given season and position
def calculate_rating(df, position_model, season, position):
    # Filter the DataFrame for the given season and position
    season_df = df[(df['Season'] == season) & (df['Position'] == position)]

    if season_df.empty:
        print(f"No data for {position} in {season}")
        return pd.DataFrame()

    # Get the feature importances
    importances = position_model.new_importance_df.set_index('Feature')['Importance']

    # Calculate the maximum minutes for the season
    max_minutes = season_df['Minutes Played'].max()

    # Calculate the weighted sum of metrics
    weighted_sum = season_df[importances.index].mul(importances, axis=1).sum(axis=1)

    # Calculate the raw rating
    season_df['Raw Rating'] = weighted_sum

    # Adjust rating based on minutes played
    season_df['Minutes Factor'] = season_df['Minutes Played'] / max_minutes
    season_df['Adjusted Rating'] = season_df['Raw Rating'] * season_df['Minutes Factor']

    # Calculate division weight
    season_df['Division Weight'] = season_df['Division'].map(division_ranks).apply(calculate_division_weight)

    # Apply division weight to the adjusted rating
    season_df['Weighted Rating'] = season_df['Adjusted Rating'] * season_df['Division Weight']

    # Scale the Weighted Rating to 0-10 (this is now our SABR Rating)
    min_rating = season_df['Weighted Rating'].min()
    max_rating = season_df['Weighted Rating'].max()
    season_df['SABR Rating'] = ((season_df['Weighted Rating'] - min_rating) / (max_rating - min_rating) * 10).round(2)

    # Sort by SABR Rating in descending order
    season_df = season_df.sort_values('SABR Rating', ascending=False)

    # Structure of the final df
    return season_df[['UID','Name', 'Position', 'Age', 'Club', 'Division', 'Minutes Played',
        'Aerial Challenges Attempted per 90',
        'Assists per 90',
        'Blocks per 90',
        'Chances Created per 90',
        'Clearances per 90',
        'Cross Completion %',
        'Dribbles per 90',
        'Expected Assists per 90',
        'Expected Goals per 90',
        'Goals per 90',
        'Headers Won %',
        'Interceptions per 90',
        'Key Headers per 90',
        'Key Passes per 90',
        'Key Tackles per 90',
        'Non-Penalty xG per 90',
        'Open-Play Cross Completion %',
        'Open-Play Key Passes per 90',
        'Possession Lost per 90',
        'Possession Won per 90',
        'Pressures Attempted per 90',
        'Pressures Completed per 90',
        'Progressive Passes per 90',
        'Shots Blocked per 90',
        'Shots Outside Box per 90',
        'Shots on Target per 90',
        'Shots per 90',
        'Tackles Won per 90',
        'Season',
        'Average Rating',
        'SABR Rating']]

# List of seasons, positions, and divisions in the df
seasons = sorted(df['Season'].unique())
positions = sorted(df['Position'].unique())
divisions = list(world['Name'])

# Dictionary to map position names to their respective models
position_models = {
    'Centre Back': centre_backs_model,
    'Full Back': full_backs_model,
    'Wing Back': wing_backs_model,
    'Defensive Midfielder': defensive_midfielders_model,
    'Central Midfielder': central_midfielders_model,
    'Central Attacking Midfielder': central_attacking_midfielders_model,
    'Winger': wingers_model,
    'Wide Attacking Midfielder': wide_attacking_midfielders_model,
    'Striker': strikers_model
}

# Check if all positions in the data have corresponding models
missing_models = set(positions) - set(position_models.keys())
if missing_models:
    print(f"Warning: The following positions do not have corresponding models: {missing_models}")

# Calculate ratings for each season, position, and division
all_ratings = []

for season in seasons:
    for position in positions:
        if position in position_models:
            print(f"Calculating ratings for {position} in season {season}")
            ratings = calculate_rating(df, position_models[position], season, position)
            if not ratings.empty:
                print(f"Calculated ratings for {len(ratings)} players in {position} for season {season}")
                all_ratings.append(ratings)
            else:
                print(f"No ratings calculated for {position} in season {season}")
        else:
            print(f"No model found for {position}")

print(f"Total number of rating calculations: {len(all_ratings)}")

# Combine all ratings into a single dataframe
if all_ratings:
    final_ratings = pd.concat(all_ratings, ignore_index=True)

    # Calculate the mean of 'Average Rating' and 'SABR Rating' for each UID
    mean_ratings = final_ratings.groupby('UID').agg({
        'Average Rating': 'mean',
        'SABR Rating': 'mean',
        'Name': 'first',  # Keep the first occurrence of the name
        'Position': lambda x: ', '.join(set(x)),  # Join unique positions
        'Age': 'max',  # Use the maximum age
        'Club': 'last',  # Use the last club
        'Division': 'last',  # Use the last division
        'Minutes Played': 'sum',  # Sum up all minutes played
        'Season': 'nunique'  # Count unique seasons
    }).reset_index()

    # Rename the columns for more clarity
    mean_ratings = mean_ratings.rename(columns={
        'Average Rating': 'Adjusted Rating',
        'SABR Rating': 'Adjusted SABR Rating',
        'Season': 'Total Number Of Seasons Stored'
    })

    # Merge the mean ratings back to the final_ratings dataframe
    final_ratings = final_ratings.merge(mean_ratings[['UID', 'Adjusted Rating', 'Adjusted SABR Rating', 'Total Number Of Seasons Stored']], on='UID')

    # Round the ratings to two decimal points
    final_ratings['SABR Rating'] = final_ratings['SABR Rating'].round(2)
    final_ratings['Adjusted Rating'] = final_ratings['Adjusted Rating'].round(2)
    final_ratings['Adjusted SABR Rating'] = final_ratings['Adjusted SABR Rating'].round(2)

    # Add the difference column
    final_ratings['Rating Difference'] = (final_ratings['Adjusted SABR Rating'] - final_ratings['Adjusted Rating']).round(2)

    # Rearrange the columns in the desired order
    column_order = [
        'UID', 'Name', 'Position', 'Age', 'Club', 'Division', 'Minutes Played',
        'Season', 'Total Number Of Seasons Stored', 'Average Rating', 'SABR Rating',
        'Adjusted Rating', 'Adjusted SABR Rating', 'Rating Difference',
        'Aerial Challenges Attempted per 90', 'Assists per 90', 'Blocks per 90',
        'Chances Created per 90', 'Clearances per 90', 'Cross Completion %',
        'Dribbles per 90', 'Expected Assists per 90', 'Expected Goals per 90',
        'Goals per 90', 'Headers Won %', 'Interceptions per 90', 'Key Headers per 90',
        'Key Passes per 90', 'Key Tackles per 90', 'Non-Penalty xG per 90',
        'Open-Play Cross Completion %', 'Open-Play Key Passes per 90',
        'Possession Lost per 90', 'Possession Won per 90', 'Pressures Attempted per 90',
        'Pressures Completed per 90', 'Progressive Passes per 90', 'Shots Blocked per 90',
        'Shots Outside Box per 90', 'Shots on Target per 90', 'Shots per 90',
        'Tackles Won per 90'
    ]

    final_ratings = final_ratings.reindex(columns=column_order)

    # Save the final ratings to a CSV file
    final_ratings.to_csv('***/player_ratings.csv', index=False)

    print("Ratings calculation complete. Results saved to CSV.")
else:
    print("No ratings were calculated.")